# Leading Indicators Jupyter Notebook 3
In Jupyter Notebook 2 it was discovered that BNS.TO had 5 of the top 10 leading indicators.  
This notbook focusses on finding the top 10 leading indicators of the BNS.TO Security.  

# Import Python Libraries

In [ ]:
# Initial imports
import os
import csv
import requests
import pandas as pd
#from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import warnings
from pathlib import Path
import datetime as dt

#Subsequent imports
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
%matplotlib inline

# Import Data - Security/Stock Prices

## 1. Yahoo Finance Data

In [ ]:
# List the ticker symbols for the stocks of interest
lst=['BMO.TO', 'BNS.TO', 'CM.TO', 'NA.TO','RY.TO', 'TD.TO', 'ZEB.TO','ZWB.TO', 'XGB.TO', 'CL' ]

# Determine the start and end dates
start_date = '2016-08-01'
end_date = '2019-07-31'

# Create empty dataframe to hold stock data
dfAllStocks_Yahoo=pd.DataFrame()

# Gathers stock data from listed tickers and combines them into a dataframe
for ticker in lst:
    # Create data frame from API in loop 
    df= yf.download(ticker, start= start_date, end= end_date)
    # Remove to '.TO' suffix from Canadian stocks
    tic= ticker.replace('.TO','')
    df['Ticker']=tic
    # Drop unwanted coloumns - Keep Adjusted Close
    df.drop(columns= ["Open", "High", "Low", "Volume", "Close", "Ticker"], inplace= True)
    # Combine individual stocks into a single data frame
    dfAllStocks_Yahoo=pd.concat([dfAllStocks_Yahoo, df], axis= 1)

# Rename the coloumns to the ticker symbols in the list
dfAllStocks_Yahoo.columns = lst
dfAllStocks_Yahoo.index.name = 'Date'

# Drop the N/As
dfAllStocks_Yahoo = dfAllStocks_Yahoo.dropna()

# Drop the time component of the date
dfAllStocks_Yahoo.index = dfAllStocks_Yahoo.index.date

# Display the data
display(dfAllStocks_Yahoo)

## 2. CSV Import - Trade Station Data

In [ ]:
# Commodities
cd_path = Path("../data/CD_10Year.txt")
dx_path = Path("../data/DX_10Year.txt")

# Commodities
cd_df = pd.read_csv(cd_path, index_col = "Date", parse_dates = True, infer_datetime_format=True)
cd_df = cd_df.drop(columns=['Time', 'Open', 'High', 'Low', 'Vol', 'OI'])
cd_df.columns = ["USDCAD"]
cd_df['USDCAD'] = cd_df['USDCAD'].copy() * 100
dx_df = pd.read_csv(dx_path, index_col = "Date", parse_dates = True, infer_datetime_format=True)
dx_df = dx_df.drop(columns=['Time', 'Open', 'High', 'Low', 'Vol', 'OI'])
dx_df.columns = ["DXY"]

# Display the data
#display(cd_df)
#display(dx_df)

## 3. Concatinate Dataframes

In [ ]:
# Join Yahoo Finance and Trade Station CSV files.
all_stocks_df = pd.concat([dfAllStocks_Yahoo, cd_df, dx_df], axis="columns", join="inner")

# Testing Purposes: Exclude CSV Files Data
#all_stocks_df=all_stocks_df.drop(columns=['CD', 'DX'])

# Ensure data sorted by index 
all_stocks_df.index.name = 'Date'
all_stocks_df.sort_index(inplace = True)

# Display sample data
all_stocks_df
#all_stocks_df.columns

# Prepare Data for Analysis

## 1. Dataframe Column Names to List

In [ ]:
lst = all_stocks_df.columns.to_list()
print(len(lst))
print(type(lst))
print(lst)
print(all_stocks_df.index.name)

## 2. Validate Nested Loops through Data Frame  
- Leader is the Security viewed as the Leading Indicator
- Target is the Security being predicted by the Leading Indicator
- Number of Paris = n(n-1) = 12 x 11 = 132 pairs of Securities

In [ ]:
# Nested loop though all securities creating pairs of securities 
"""
for leader in lst:
    count = 0
    lst2 = lst.copy()
    lst2.remove(leader)       # Exclude pairs of same security
    for target in lst2:
        print(leader + ' - ' + target)
        count += 1
    print(f'Number of {leader} Security Pairs : {count}. \n')
"""

## 3. Analysis Routine - Looking for a relationship between the Leader and Target

In [ ]:
all_stocks_df

In [ ]:
# Loop through all securities as a leading indicator:
index_count = 0
df_correlation = pd.DataFrame(columns = ['p', 'Leader', 'Target', 'Corr', 'AbsCorr'] )
lst = all_stocks_df.columns.to_list()
#lst = lst.remove("BNS.TO")

for leader in lst:
    lst2 = lst.copy()
    lst2.remove(leader)
    lst2 = ['BNS.TO']
    
# Loop through all stosck excluding the lerading indicator:    
    for target in lst2:

# Create and empty DataFrame for closing prices
        df_closing_prices = pd.DataFrame()
        

# Fetch the closing prices of Stock being lead
        df_closing_prices[leader] = all_stocks_df[leader]
        df_closing_prices[target] = all_stocks_df[target]

# Calcualte for Leader Stock
        #df_closing_prices['leader_moving_average']=df_closing_prices[leader].rolling(window=5).mean()
        #df_closing_prices['leader_tstat']=(df_closing_prices[leader]-df_closing_prices['leader_moving_average'])/df_closing_prices[leader].rolling(window=200).std()
        #df_closing_prices['leader_pct_chg']=df_closing_prices[leader].pct_change()
        #df_closing_prices['target_moving_average']=df_closing_prices[target].rolling(window=5).mean()

# Drop columns not required
        #df_closing_prices=df_closing_prices.drop(columns=['leader_moving_average'])
        #display(df_closing_prices)
        df_closing_prices=df_closing_prices.dropna()
        #display(df_closing_prices)

# Loop through varios Leading Indicator Scenarios:
        for p in range(1,30,5):
            print('***************************************************************************************************')
            print(f' Iteration {p} for Leader: {leader} vs. Target: {target}')
            print('***************************************************************************************************')
            dfopt=df_closing_prices
            dfopt['Target_Change']=dfopt[target].pct_change(p).shift(-p)
            dfopt['Leader_Change']=dfopt[leader].pct_change(p)
            dfopt=dfopt[['Target_Change','Leader_Change']]
            dfcorr = dfopt.corr() 
            display(dfopt.corr())
            dfopt=dfopt.dropna()
            dfopt['RtnQuantile']=pd.qcut(dfopt['Target_Change'], 5,labels = False)
            dfPlot=dfopt.groupby('RtnQuantile')[['Leader_Change']].mean()
            #display(dfPlot)
            display(dfPlot.hvplot.bar())
            display(dfopt.hvplot.scatter(y='Target_Change',x='Leader_Change'))
            #display(df_closing_prices.hvplot.scatter(y='FutureChange',x=tic_tstat))

# Put correlation into a dataframe            
            df_temp = pd.DataFrame({"p":p, "Leader":leader, "Target":target, "Corr":dfcorr.iloc[0][1], "AbsCorr": abs(dfcorr.iloc[0][1])}, index = [index_count])
            df_correlation = pd.concat([df_correlation,df_temp], axis = "rows")
            index_count += 1
            
        del(dfopt)
        del(dfPlot)


In [ ]:
df_correlation.drop(df_correlation[(df_correlation['Leader'] == 'BNS.TO')].index, inplace=True)
display(df_correlation.sort_values('AbsCorr', ascending=[False]).head(10))